# How does population look like without the impact of the HIV/AIDS epidemic?
Modeling and Simulation - Project 1

Skye Ozga, HK Rho

### Reading the Data

In [2]:
#######NEW PROJECT########
#QUESTION: compare the actual data graph population to the graph without the HIV/AIDS epidemic 
#          so that we can see by what factor the population differs in two situations

import random
import pandas as pd

# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim library
from modsim import *

import numpy as np

def mean(x,y):
    return (x+y)/2

population change, birth rate, death rate change due to the aids epidemic in Sub-Saharan Africa

Question:
By what factor will the population change if the effects of the HIV/AIDS epidemic are disregarded compared to the actual population graph?

Methodology/Model:
The deaths caused by HIV/AIDS can be disregarded from the overall death rate of the entire population of the country we are analyzing, so that we can compare two situations with and without HIV/AIDS.

Results:
There is no results yet, but the results will consist of two graphs that each show the population change with and without HIV/AIDS.

Interpretation:
After generating the graphs, the amount by which one differs from the other can be observed/calculated.

In [ ]:
#DA = Deaths due to AIDS of all ages
#AIDS years: 1990-2017
angolaDA = pd.read_csv("angolaDA.csv", usecols = [1])
zambiaDA = pd.read_csv("zambiaDA.csv", usecols = [1])
malawiDA = pd.read_csv("malawiDA.csv", usecols = [1])
mozambiqueDA = pd.read_csv("mozambiqueDA.csv", usecols = [1])
zimbabweDA = pd.read_csv("zimbabweDA.csv", usecols = [1])
namibiaDA = pd.read_csv("namibiaDA.csv", usecols = [1])
botswanaDA = pd.read_csv("botswanaDA.csv", usecols = [1])
eswatiniDA = pd.read_csv("eswatiniDA.csv", usecols = [1])
lesothoDA = pd.read_csv("lesothoDA.csv", usecols = [1])
sAfricaDA = pd.read_csv("sAfricaDA.csv", usecols = [1])

#population from 2000-2017
angolaPOP = [16440924, 16983266, 17572649, 18203369, 18865716, 19552542, 20262399, 20997687, 21759420, 22549547, 23369131, 
             24218565, 25096150,25998340, 27859305, 28813463, 29784193]

#BR = birth rate
#it's the birth rate per 1000 people
angolaBR =      [49.0, 48.8, 48.5, 48.3, 48.0, 47.7, 47.4, 46.9, 46.5, 45.9, 45.3, 44.7, 44.1, 43.4, 42.9, 42.3, 41.8]

#DR = death rate
#it's the death rate per 1000 people
angolaDR = [17.9, 17.0, 16.2, 15.4, 14.5, 13.8, 13.0, 12.3, 11.7, 11.0, 10.5, 9.9, 9.5, 9.2, 8.9, 8.7, 8.5]

years = [x for x in range(2000,2017)];

In [ ]:
#plot angola pop data
plot(years,angolaPOP)

In [ ]:
#plot angola birthrate
plot(years,angolaBR)

In [ ]:
#plot angola deathrate
plot(years,angolaDR)

In [ ]:
#this is the revised death rate without the impact of the HIV/AIDS epidemic

#numDeaths = DR * pop
#newDR = (numDeaths - DA) / (pop + DA)
#and then use the newDR to model a population without HIV/AIDS


#this is the revised birth rate without the impact of the HIV/AIdS epidemic

#a is a random number which a% of the DA people give birth 
#a = random.randit(0,1)

#numBirths = BR * pop
#newBR = (a * numBirths) / (pop + DA)